In [ ]:
def sat_display(filenc,feature,style="gray",index=0,crop=True,scale=True,domain=True,colorbar=True,savefig=False):
    """
    filegrb: a string containing the filepath to the file that ends in .grb or .grb2
    feature: a string containing the feature to be displayed
    style: a string containing the cmap coloration
    index: an integer used to denote several versions of the feature variable
    crop: a boolean variable used to show the cropped image
    scale: a boolean variable used to scale the data down between 0 and 1
    """
    if (crop == True):
        from netCDF4 import Dataset
        import matplotlib.pyplot as plt
        import numpy as np
        nc_fid = Dataset(filenc, 'r') # "opening" the satellite data file 
        #print(nc_fid) # this should print out all the information of the file, similar to a pandas dataframe
        data = nc_fid.variables['data'][:][0] # extracting the data we want from the file
        lat = nc_fid.variables['lat'][:]
        lon = nc_fid.variables['lon'][:]
        nc_fid.close() # closing the file after using it
        lat_bot = 31.0
        lat_top = 41.0
        lon_left = -102.0
        lon_right = -82.0
        data[lat < lat_bot] = 0
        data[lat > lat_top] = 0
        data[lon < lon_left] = 0
        data[lon > lon_right] = 0
        lat = lat[data != 0]
        lon = lon[data != 0]
        data = data[data != 0]
        print(data.min())
        if scale==True:
            data = (data/data.max())
        plt.scatter(x=lon,y=lat,s=data,c=data,marker=',',cmap=style)
        plt.title(feature + " of Domain")
        plt.xlabel("Longitude W")
        plt.ylabel("Latitude N")
        if colorbar==True:
            plt.colorbar()
        if savefig==True:
            plt.title("")
            plt.xlabel("")
            plt.ylabel("")
            plt.savefig("foo.png",bbox_inches="tight")
    else:
        from netCDF4 import Dataset
        import matplotlib.pyplot as plt
        import numpy as np
        nc_fid = Dataset(filenc, 'r') # "opening" the satellite data file 
        #print(nc_fid) # this should print out all the information of the file, similar to a pandas dataframe
        data = nc_fid.variables['data'][:][0] # extracting the data we want from the file
        lat = nc_fid.variables['lat'][:]
        lon = nc_fid.variables['lon'][:]
        nc_fid.close()
        if scale==True:
            data = (data/data.max())
        if domain==True:
            data[lat < 31.0] = 0
            data[lat > 41.0] = 0
            data[lon < -102.0] = 0
            data[lon > -82.0] = 0
        plt.imshow(data,cmap=style)
        plt.title(feature + " of NA")
        plt.xlabel("Longitude W")
        plt.ylabel("Latitude N")
        if colorbar==True:
            plt.colorbar()

In [ ]:
def mod_display(filegrb,feature,style="gray",index=0,crop=True,scale=True,domain=True,colorbar=True):
    """
    filegrb: a string containing the filepath to the file that ends in .grb or .grb2
    feature: a string containing the feature to be displayed
    style: a string containing the cmap coloration
    index: an integer used to denote several versions of the feature variable
    crop: a boolean variable used to show the cropped image
    scale: a boolean variable used to scale the data down between 0 and 1
    """
    if (crop == True):
        import pygrib
        import matplotlib.pyplot as plt
        import numpy as np
        lat_bot = 31.0
        lat_top = 41.0
        lon_left = -102.0
        lon_right = -82.0
        grbs = pygrib.open(filegrb)
        grb = grbs.select(name=feature)[index]
        grbs.close()
        data,lats,lons = grb.data(lat1=lat_bot,lat2=lat_top,lon1=lon_left,lon2=lon_right)
        if scale==True:
            data = data+abs(data.min())
            data = (data/data.max()+1)*10
        print(data.min())
        plt.scatter(x=lons,y=lats,s=data,c=data,marker=",",cmap=style)
        plt.title(feature + " of Domain")
        plt.xlabel("Longitude W")
        plt.ylabel("Latitude N")
        if colorbar==True:
            plt.colorbar()
    else:
        import pygrib
        import matplotlib.pyplot as plt
        import numpy as np
        grbs = pygrib.open(filegrb)
        grb = grbs.select(name=feature)[index]
        grbs.close()
        data = grb.values
        if scale==True:
            data = (data/data.max())
        if domain==True:
            lats,lons = grb.latlons()
            data[lats < 31.0] = np.nan
            data[lats > 41.0] = np.nan
            data[lons < -102.0] = np.nan
            data[lons > -82.0] = np.nan
        plt.imshow(data,cmap=style,origin="lower")
        plt.title(feature + " of NA")
        plt.xlabel("Longitude W")
        plt.ylabel("Latitude N")
        if colorbar==True:
            plt.colorbar()

In [ ]:
filegrb = "/mnt/data-weather/moddata/modgrb/namanl_218_20170102_1200_002.grb"
filenc = "/mnt/data-weather/satdata/goes13.2017.002.141518.BAND_02.nc"
feature = "Temperature"

In [ ]:
sat_display(filenc,feature="clouds",crop=False,scale=False,style='Greys',domain=False)

In [ ]:
sat_display(filenc,"clouds",crop=False,scale=False,style='Greys',domain=True)

In [ ]:
sat_display(filenc,"clouds",crop=True,scale=True,style='Greys',colorbar=True,savefig=False)

In [ ]:
mod_display(filegrb,feature,crop=False)

In [ ]:
import pygrib
grbs = pygrib.open(filegrb)
for grb in grbs:
    print(grb)
grb=grbs.select(name="Temperature")[0]
print(grb.values.max(),grb.values.min())
grb=grbs.select(name="Soil Temperature")[0]
print(grb.values.max(),grb.values.min())
grb=grbs.select(name="Total Precipitation")[0]
print(grb.values.max(),grb.values.min())
grb=grbs.select(name="Convective precipitation (water)")[0]
print(grb.values.max(),grb.values.min())
grb=grbs.select(name="Specific humidity")[0]
print(grb.values.max(),grb.values.min())
grb=grbs.select(name="Relative humidity")[0]
print(grb.values.max(),grb.values.min())
grb=grbs.select(name="Visibility")[0]
print(grb.values.max(),grb.values.min())
grb=grbs.select(name="Visibility")[1]
print(grb.values.max(),grb.values.min())


grbs.close()

In [ ]:
mod_display(filegrb,"Soil Temperature",crop=False,domain=False,colorbar=True,scale=False)

In [ ]:
mod_display(filegrb,"Soil Temperature",crop=False,domain=True,colorbar=True,scale=False)

In [ ]:
mod_display(filegrb,"Soil Temperature",crop=True,colorbar=True,scale=False,style='gray')

In [ ]:
mod_display(filegrb,"Relative humidity",crop=False,domain=False,colorbar=True,scale=False,index=0,style='gray')

In [ ]:
mod_display(filegrb,"Relative humidity",crop=False,domain=True,colorbar=True,scale=False,style='gray')

In [ ]:
mod_display(filegrb,"Relative humidity",crop=True,colorbar=True,scale=False,style='gray')

In [ ]:
mod_display(filegrb,"Geopotential Height")